In [8]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path

project_root = Path.cwd()
sys.path.insert(0, str(project_root))

from prepare_params import prepare_production_params

print("Modules imported successfully")
print(f"Current working directory: {os.getcwd()}")


kml = "Sample Points\AfforestationSItesFixed.kml"
shp = "Sample Points\FieldPoints32_2018.shp"
tmx = "Sample Points\TMX\TML_pipeline_100mbuffer.kml"
cbc = "Sample Points\ColdwaterBCregion.kml"

print(kml)
print(shp)
print(tmx)
print(cbc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Modules imported successfully
Current working directory: C:\Users\aradar\LEAF Files\LEAF_runner
Sample Points\AfforestationSItesFixed.kml
Sample Points\FieldPoints32_2018.shp
Sample Points\TMX\TML_pipeline_100mbuffer.kml
Sample Points\ColdwaterBCregion.kml


In [9]:
# TEMPLATE 1: Single Year Monthly Mosaics
# Use Case: Generate monthly mosaics for specific months in one year
# Example: Summer months (June, July, August) of 2023
# Customize these parameters as needed:

ProdParams = {
    # ============ REQUIRED PARAMETERS ============
    'sensor': 'S2_SR',                    # Sensor type
    'unit': 2,                   # A data unit code (1 or 2 for TOA or surface reflectance)    
    'nbYears': -1,            # positive int for annual product, or negative int for monthly product
    #'year': 2023,                          # CHANGE THIS: Year to process
    #'months': [8,9,10],                   # CHANGE THIS: Months to process (1-12)
    
    
    # ============ REGION PARAMETERS ============
    'regions': kml,  # CHANGE THIS: Path to your KML/SHP file
    'file_variables': ['TARGET_FID', 'AsssD_1', 'AsssD_2'], # CHANGE THIS: id, start_date, end_date (None for dates if they don't exist)
    #'file_variables': ['id', 'begin', 'end'],
    #'file_variables': ['system:index', None, None],
    'regions_start_index': 3000,              # CHANGE THIS: Start at this region index
    'regions_end_index': 3999,             # CHANGE THIS: End at this index (None = all)
    
    # ============ BUFFER PARAMETERS ============
    #'spatial_buffer_m': 0,             # UNCOMMENT & CHANGE: Buffer in meters around regions
    #'temporal_buffer': [[50, 90], [-10, 10]],          # UNCOMMENT & CHANGE: [days_before, days_after]
    'temporal_buffer': [["2025-08-01", "2025-08-31"]],
    #'num_years': 3,
    
    
    # ============ OUTPUT PARAMETERS ============
    'resolution': 20,                      # Resolution in meters
    'projection': 'EPSG:3979',             # Coordinate projection
    'prod_names': ['mosaic'],    #['LAI', 'FCOVER', 'fAPAR', 'Albedo'], 
    #'bands': ['blue', 'green', 'red', 'rededge1', 'rededge2', 'rededge3', 'nir08', 'swir16', 'swir22'],
    'out_folder': 'E:/S2_mosaics_runner_2026/benchmark/S2_3000-3999_august_2025_10m',
    'out_datatype': 'int16'
}

CompParams = {
  "number_workers":32,     
  "debug"       : True,
  "entire_tile" : False, 
  "nodes"       : 1,
  "node_memory" : "16G",
  'chunk_size': {'x': 512, 'y': 512}
}


print("\nconfigured")


configured


In [10]:
# Run the parameter preparation
result = prepare_production_params(ProdParams, CompParams)
'''
if result:
    print("\n" + "="*80)
    print(" SUCCESS - Parameters validated and ready!")
    print("="*80)
    print(f"\n Summary:")
    print(f"  Regions: {len(result['ProdParams']['regions'])}")
    print(f"  Time windows: {len(result['ProdParams']['start_dates'])}")
    print(f"  Total tasks: {len(result['ProdParams']['regions']) * len(result['ProdParams']['start_dates'])}")
    print(f"  Output folder: {result['ProdParams']['out_folder']}")
    print(f"\n Processing log saved to: {result['ProdParams']['out_folder']}/polygon_processing_log.csv")
    print("\n" + "="*80)
    print("Next step: Call Production.py with the validated parameters")
    print("="*80)
else:
    print("\n" + "="*80)
    print(" FAILED - Parameter validation failed")
    print("="*80)
    print("Check the error messages above for details.") '''


PARAMETER PREPARATION

Step 1/4: Validating input parameters...
✓ All parameters valid

Step 2/4: Loading regions from files...
<handle_regions_from_file> Detected file-based regions input: Sample Points\AfforestationSItesFixed.kml
<handle_regions_from_file> Loading regions...
<handle_regions_from_file> Region region3526: Auto-generated end dates from start dates
<handle_regions_from_file> Region region3544: Auto-generated end dates from start dates
<handle_regions_from_file> Region region3529: Auto-generated end dates from start dates
<handle_regions_from_file> Region region3924: Auto-generated start dates from end dates
<handle_regions_from_file> Region region3803: Auto-generated end dates from start dates
<handle_regions_from_file> Region region3843: Auto-generated start dates from end dates
<handle_regions_from_file> Region region3832: Auto-generated start dates from end dates
<handle_regions_from_file> Region region3364: Auto-generated end dates from start dates
<handle_regions_f

'\nif result:\n    print("\n" + "="*80)\n    print(" SUCCESS - Parameters validated and ready!")\n    print("="*80)\n    print(f"\n Summary:")\n    print(f"  Regions: {len(result[\'ProdParams\'][\'regions\'])}")\n    print(f"  Time windows: {len(result[\'ProdParams\'][\'start_dates\'])}")\n    print(f"  Total tasks: {len(result[\'ProdParams\'][\'regions\']) * len(result[\'ProdParams\'][\'start_dates\'])}")\n    print(f"  Output folder: {result[\'ProdParams\'][\'out_folder\']}")\n    print(f"\n Processing log saved to: {result[\'ProdParams\'][\'out_folder\']}/polygon_processing_log.csv")\n    print("\n" + "="*80)\n    print("Next step: Call Production.py with the validated parameters")\n    print("="*80)\nelse:\n    print("\n" + "="*80)\n    print(" FAILED - Parameter validation failed")\n    print("="*80)\n    print("Check the error messages above for details.") '

In [11]:
# View processing log (if pandas is available)
try:
    import pandas as pd
    if result and result['processing_log'] is not None:
        print("\n Processing Log Preview:")
        print("=" * 80)
        display(result['processing_log'].head(10))
        print(f"\nTotal entries: {len(result['processing_log'])}")
except ImportError:
    print("pandas not available - install it to view the processing log in this cell")
    print("You can still view the CSV file directly in the output folder")


 Processing Log Preview:


,region_id,area_m2,will_process,status,skip_reason
0,region3000,44774.88,True,VALID,N/A
1,region3001,10896.01,True,VALID,N/A
2,region3002,18182.65,True,VALID,N/A
3,region3003,37156.03,True,VALID,N/A
4,region3004,23785.18,True,VALID,N/A
5,region3005,181501.76,True,VALID,N/A
6,region3006,33550.50,True,VALID,N/A
7,region3007,18452.97,True,VALID,N/A
8,region3008,18422.45,True,VALID,N/A
9,region3009,53098.11,True,VALID,N/A



Total entries: 1000


In [12]:
import os
import sys
import geopandas as gpd

#Get the absolute path to the parent of current working directory 
cwd    = os.getcwd()
source_path = os.path.join(cwd, 'source')
sys.path.append(source_path)
sys.path

from Production import main

print(ProdParams)

{'sensor': 'S2_SR', 'unit': 2, 'nbYears': -1, 'regions': {'region3000': {'type': 'Polygon', 'coordinates': [[[-79.6670620817455, 42.8602791217525], [-79.6646034950008, 42.8603031575409], [-79.6645579855655, 42.8582397369697], [-79.6669475253634, 42.8582753453209], [-79.6670620817455, 42.8602791217525]]]}, 'region3001': {'type': 'Polygon', 'coordinates': [[[-79.8791581476637, 43.0012500029038], [-79.8790381481434, 43.0007960024444], [-79.8792031470735, 43.0005190025837], [-79.8797991473824, 43.0004270024094], [-79.8805951478391, 42.9991660031282], [-79.8811881478653, 42.9988570028293], [-79.8818781475824, 42.998997002649], [-79.8810941475347, 42.9996240027728], [-79.8805191481314, 42.9995100024251], [-79.8801821476792, 43.0000080023973], [-79.8799771472926, 43.000333002788], [-79.8800731477599, 43.0006380030711], [-79.8791581476637, 43.0012500029038]]]}, 'region3002': {'type': 'Polygon', 'coordinates': [[[-79.5204111487308, 43.1100380034641], [-79.5191531485078, 43.1130960031875], [-79.

In [ ]:
main(ProdParams, CompParams)


<get_time_window> one of required keys is not exist to form time.
<MosaicProduction> User defined input parameters:
sensor: S2_SR
unit: 2
nbYears: -1
regions: {'region3000': {'type': 'Polygon', 'coordinates': [[[-79.6670620817455, 42.8602791217525], [-79.6646034950008, 42.8603031575409], [-79.6645579855655, 42.8582397369697], [-79.6669475253634, 42.8582753453209], [-79.6670620817455, 42.8602791217525]]]}, 'region3001': {'type': 'Polygon', 'coordinates': [[[-79.8791581476637, 43.0012500029038], [-79.8790381481434, 43.0007960024444], [-79.8792031470735, 43.0005190025837], [-79.8797991473824, 43.0004270024094], [-79.8805951478391, 42.9991660031282], [-79.8811881478653, 42.9988570028293], [-79.8818781475824, 42.998997002649], [-79.8810941475347, 42.9996240027728], [-79.8805191481314, 42.9995100024251], [-79.8801821476792, 43.0000080023973], [-79.8799771472926, 43.000333002788], [-79.8800731477599, 43.0006380030711], [-79.8791581476637, 43.0012500029038]]]}, 'region3002': {'type': 'Polygon

## 